In [3]:
import os, glob
import pandas as pd
import subprocess
from functools import reduce
from biom import load_table
import multiprocessing as mp
import datetime
import itertools
import numpy as np
import scipy
from fitter import Fitter
from scipy import stats
from scipy.spatial import distance

# import random

# from memory_profiler import profile
# from memory_profiler import memory_usage

# %reload_ext memory_profiler


In [4]:
#from functs import remove_per_paper,get_marginals,get_sparsity,get_span_countProp,nskip_header,get_f_dist,get_feats_fits,get_distrib_freqs_per_feature,get_distrib_freqs_vec,get_kl

# Functions

In [63]:
# for the parada17 paper - always remove the samples which have "DOBD" in mock_or_monthlies"
def remove_per_paper(table_pd, meta_pd):
    """
    For the passed table, treat specifically depending on the qiita_paper:
        - if there is a "misc environment" in the metadata for this dataset:
            -> remove the BLANKS / CONTROLS / MOCKS by keeping assays
    Returns the filtered table
    """
    if 'misc environment' in meta_pd.env_package.unique():
        samples_to_keep = set(meta_pd.loc[meta_pd['env_package'] == 'misc environment',:].index.tolist())
        table_pd = table_pd.loc[:, [x for x in table_pd.columns if '.'.join(x.split('.')[1:]) not in samples_to_keep]]
        table_pd = table_pd[(table_pd.T != 0).any()]
    else:
        print('no blank/mock')
    return table_pd

def get_marginals(tab):
    """
    For the passed table, get:
        - the sums of reads per sample (must be columns)
        - the sums of reads per features (must be rows)
    Returns (min, max) for both
    """
    samplesSums = round(tab.sum(0), 3)
    featuresSums = round(tab.sum(1), 3)
    return (min(featuresSums), max(featuresSums)), (min(samplesSums), max(samplesSums))

def get_sparsity(tab):
    """
    For the passed table, get:
        - total number of entries
        - total number of zeros
        - total number of ones
        - sparsity (as percent zeros)
    Returns these four properties
    """
    NN = tab.shape[0] * tab.shape[1]
    ZR = (tab==0).sum().sum()
    ONE = (tab==1).sum().sum()
    sparsity = round((ZR/NN)*100, 3)
    return NN, ZR, ONE, sparsity

def get_span_countProp(tab, min_max_samples_sums):
    """
    Checks if binary and if not get the actual values
      binary_count_or_prop could be:
       - [0,1] for binary
       - [n1,n2,...n10] for count (.hist())
       - [0.,1.] or [0.,100.] for prop
    Returns count and type
    """
    # set data_type as 'float' or 'int' if all the columns are integers
    data_type = 'float'
    int_cols = tab.apply(lambda x: pd.to_numeric(x, errors='coerce').notnull().all())
    if int_cols.sum() == tab.shape[1]:
        data_type = 'int'

    # if the data_type is still 'float' 
    if data_type == 'float':
        if min(min_max_samples_sums) >= 0:
            if min(min_max_samples_sums) <= 1.:
                return [0., 1.], 'prop'
            elif min(min_max_samples_sums) <= 100.:
                return [0., 100.], 'prop'
            else:
                return [0., min_max_samples_sums[1]], 'prop'
        
    # get the frequencies if the table in 10 bins
    count, division = np.histogram(tab)
    return list(count), 'count'

def nskip_header(tab):
    """
    Checks whether the passed table starts
        by the biom convert --to-tsv header
    Returns 0 if no, 1 if yes
    """
    with open(tab) as f:
        for line in f:
            if line.startswith('# Constructed from biom file'):
                return 1
            else:
                return 0
            

def get_f_dist(cur_pd, feat, dist_names):
    """
    For the passed feature,
     get the sumsquare_error of the
     fit on each distribution as dict:
        - key   = distribution name
        - value = sumsqure_error
    Returns this dict
    """
    f = Fitter(cur_pd.loc[feat,], distributions=dist_names, verbose=False)
    f.fit()
    f_dist = f.summary(Nbest=len(dist_names)).to_dict()['sumsquare_error']
    return f_dist

def get_feats_fits(cur_pd, dist_names):
    """
    For the passed table, get a dict:
        - key   = feature names
        - value = sumsquare errors for each distribution
    Returns the dict
    """
    # get the features names
    cur_pd_list = cur_pd.index.tolist()

    ## get 100 randomly picked features
    # cur_pd_list = random.sample(cur_pd_list,10)
    
    # for each feature, fill list with the SSE on each distribution
    feats_fits = dict([x, []] for x in dist_names)
    for fdx, feat in enumerate(cur_pd_list):


        # if fdx == 10:
        #     break


        # get the dict {'distrib': 'SSE'}  for the current feature
        f_dist = get_f_dist(cur_pd, feat, dist_names)
        # for each distribution, 
        for dist in dist_names:
            feats_fits[dist].append(f_dist[dist])
    return feats_fits

def get_distrib_freqs_per_feature(feats_fits):
    """
    For the passed feature -> SSEs dict get:
        - the 2D array or 0 to 1 rows for each feature
    Returns this array as a pandas dataframe
    """
    # init list and read dict as table
    df_frqs = []
    df = pd.DataFrame(feats_fits)
    # for each feature vectoir of distribution errors
    for L in df.values:
        # get the mini and maxi error
        mini, maxi = min(L), max(L)
        # bin these errors along a 10-bins logspace
        lin = np.logspace(np.log10(mini), np.log10(maxi), num=10)
        dig = np.digitize(L,lin)
        # get the minimum bin value (best / first rank)
        min_rank = min(dig)
        # make ties for this rank
        n1 = list(dig).count(min_rank)
        if n1 > 1:
            v = 1/n1
            df_frq = [v if x == min_rank else 0 for x in dig]
        else:
            df_frq = [1 if x == min_rank else 0 for x in dig]
        # collect the frequencies that sum to one for the best distribution(s)
        df_frqs.append(df_frq)
    df_frqs_pd = pd.DataFrame(df_frqs)
    return df_frqs_pd

def get_distrib_freqs_vec(df_frqs_pd):
    """
    For the passed distribution ranking table,
     get the probability for each distribution across features
    Returns the 1D vector of distribution probabilities
    """
    # make the sum of the first ranks per columm
    # (i.e. frequencies of distribution across features)
    df_frqs_sum = df_frqs_pd.sum(0)
    # make it a probability
    df_frqs_sum_p = df_frqs_sum/df_frqs_sum.sum()
    return df_frqs_sum_p.tolist()

def get_distrib_freqs_data(tab, dist_names):
    # get the dict with for each feature the vector SSE to above distributions
    feats_fits = get_feats_fits(tab, dist_names)
    # get the 2D array with 0 to 1 rank for the best distribution(s) per feature
    distrib_freqs = get_distrib_freqs_per_feature(feats_fits)
    # get the 1D vector of probabilities for each distribution
    distrib_freqs_vec = get_distrib_freqs_vec(distrib_freqs)
    return distrib_freqs_vec

def get_meta_pd(table):
    # collect metadata for the qiita papers tables (not yet used)
    if '/qiita_papers/' in table:
        paper = table.split('/')[-4]
        artifact = table.split('/')[-2]
        meta_fp = '%s/qiita/%s/qiita_processed/mapping_files/%s_mapping_file.txt' % (table.split('/datasets/qiita_papers/')[0], paper, artifact)
        meta_pd = pd.read_csv(meta_fp, header=0, index_col=0, sep='\t')
        return meta_pd
    else:
        return 0

def collect_1_2_3(props_per_dataset, min_max_features_sums, min_max_samples_sums,
                  n_entries, one_entries, zero_entries, zero_entries_perc,
                  number_span, binary_count_or_prop):
    """
    Just populate a dict
    """
    props_per_dataset.setdefault('min_features_sums', []).append(min_max_features_sums[0])
    props_per_dataset.setdefault('max_features_sums', []).append(min_max_features_sums[1])
    props_per_dataset.setdefault('min_samples_sums', []).append(min_max_samples_sums[0])
    props_per_dataset.setdefault('max_samples_sums', []).append(min_max_samples_sums[1])
    props_per_dataset.setdefault('n_entries', []).append(n_entries)
    props_per_dataset.setdefault('one_entries', []).append(one_entries)
    props_per_dataset.setdefault('zero_entries', []).append(zero_entries)
    props_per_dataset.setdefault('zero_entries_perc', []).append(zero_entries_perc)
    props_per_dataset.setdefault('number_span', []).append('-'.join(map(str, number_span)))
    props_per_dataset.setdefault('binary_count_or_prop', []).append(binary_count_or_prop)
    return props_per_dataset

def write_basic(props_per_dataset, out_prop_fp, tables_chunk):
    """
    Writter
    """
    # make a main table and add the file column
    props_per_dataset_pd = pd.DataFrame(props_per_dataset)
    props_per_dataset_pd['file'] = tables_chunk
    props_per_dataset_pd.to_csv(out_prop_fp, index=False, sep='\t')
    print('Written:', out_prop_fp)

def write_distrbutions(distrib_freqs_per_dataset, dist_names,
                       tables_chunk, addon, out_prop_JS_fp):
    """
    Another writter that may append
    """
    if len(distrib_freqs_per_dataset):
        # make a pandas dataframe from all the probability vectors snd add the file column
        distrib_freqs_per_dataset_pd = pd.DataFrame(distrib_freqs_per_dataset, columns=dist_names)
        distrib_freqs_per_dataset_pd['file'] = tables_chunk
        # write only the lines for the files not already measured
        if addon:
            # read the already written file and concatenate the new neasures to it
            distrib_freqs_per_dataset_pd_0 = pd.read_csv(out_prop_JS_fp, header=0, sep='\t')
            distrib_freqs_per_dataset_pd_1 = pd.concat([distrib_freqs_per_dataset_pd_0, distrib_freqs_per_dataset_pd])
            distrib_freqs_per_dataset_pd_1.to_csv(out_prop_JS_fp, index=False, sep='\t')
        # write entire output if all distribution probability measured de novo
        else:
            distrib_freqs_per_dataset_pd.to_csv(out_prop_JS_fp, index=False, sep='\t')
    print('Written:', out_prop_JS_fp)

def run_prop_catcher(tdx, tables_chunk, tables_chunk_toJS,
                     out_prop_fp, out_prop_JS_fp, addon):
    """
    This is the nain worker that call functions for each property to collect:
        (1) marginal sums ((min, max) for both samples and features)
        (2) sparsity (number of entries, number of zeros, number of ones, percent zeros)
        (3) read counts span and data_type
        (4) feature distribtion sumsquare_errors
    """
    
    # prepare a dict that will contain other properties
    props_per_dataset = {}
    # prepare a list that will contain the lists of distributions probabilities for J-S
    distrib_freqs_per_dataset = []

    # for each tsv'ed dataset file
    for table in tables_chunk:
        meta_pd = get_meta_pd(table)
        # read the dataset (may skip first row if converted from BIOM)
        tab = pd.read_csv(table, skiprows=nskip_header(table), header=0, index_col=0, sep='\t')
        if 'qiita_paper' in table:
            tab = remove_per_paper(table_pd, meta_pd)
        
        # (1) get marginal sums
        min_max_features_sums, min_max_samples_sums = get_marginals(tab)
        # (2) get sparsity 
        n_entries, zero_entries, one_entries, zero_entries_perc = get_sparsity(tab)
        # (3) get span of read counts and string on the type of data
        number_span, binary_count_or_prop = [0,1], 'binary'
        if (zero_entries + one_entries) != n_entries:
            number_span, binary_count_or_prop = get_span_countProp(tab, min_max_samples_sums)
        
        # collect the (1) (2) and (3) properties in a dict
        props_per_dataset = collect_1_2_3(props_per_dataset, min_max_features_sums,
                                          min_max_samples_sums, n_entries, one_entries,
                                          zero_entries, zero_entries_perc,
                                          number_span, binary_count_or_prop)
        
        # (4) feature distribtion sumsquare_errors
        # distributions to look for features probabilities 


        # dist_names = ['cauchy', 'chi2', 'cosine'] 


        dist_names = ['cauchy', 'chi2', 'cosine', 'expon',
                      'gamma', 'gausshyper', 'genhalflogistic',
                      'gompertz', 'halfcauchy', 'halflogistic',
                      'halfnorm', 'logistic', 'lognorm',
                      'nakagami', 'norm', 'pareto', 'powerlaw',
                      'semicircular', 't', 'triang',
                      'tukeylambda', 'uniform']
        if os.path.isfile(out_prop_JS_fp):
            with open(out_prop_JS_fp) as f:
                for line in f:
                    break
            if dist_names != line.split()[:-1]:
                print('Cannot concatenate different distance lists: rewritting')
                addon = 0
                distrib_freqs_vec = get_distrib_freqs_data(tab, dist_names)
            elif tab not in tables_chunk_toJS:
                distrib_freqs_vec = get_distrib_freqs_data(tab, dist_names)
            else:
                continue
        else:
            distrib_freqs_vec = get_distrib_freqs_data(tab, dist_names)
        distrib_freqs_per_dataset.append(distrib_freqs_vec)

    ## write basic properties:
    write_basic(props_per_dataset, out_prop_fp, tables_chunk)
    ## features distribution properties:
    write_distrbutions(distrib_freqs_per_dataset, dist_names,
                       tables_chunk, addon, out_prop_JS_fp)

# Collect tables

In [64]:
#append the tsv'ed biome tables of each dataset into a list
tables_list = []
ntabs = 0
for root, dirs, files in os.walk('/home/flejzerowicz/netbench/datasets'):
    for fil in files:
        # skip non-tsv'ed biom tables
        if 'explanation' in fil:
            continue
        # only look into the actual folders containing tables
        if 'qiita_papers' not in root and 'weiss_paper' not in root:
            continue
        # make shure it's a tsv (here txt, dang!)
        if fil.split('.')[-1] == 'txt':
            tab = root + '/' + fil
            paper_or_weiss = '/'.join(root.split('/datasets/')[-1].split('/')[:3])
            ntabs += 1
            tables_list.append(tab)

# splits this list into a list of 12 lists corresponding to chunks of files
tables_chunks = [list(x) for x in np.array_split(np.array(sorted(tables_list)[::-1]), 12)]

# Run

In [66]:
def run(tables_chunks):
    """
    This function spawns a process for each files chunk ("ck_#") using multiprocessing
    Each process runs the worker function "run_prop_catcher":
        - it write files which names are created here in order to be collected and merged later:
            * trivial dataset properties:
                /home/flejzerowicz/netbench/datasets/properties/ck_#/tab_props.tsv
            * features distribution for Jensen-Shannon:
                /home/flejzerowicz/netbench/datasets/properties/ck_#/JS.tsv
    Returns the list of files containing the per-chunk distrtibutions for Jensen-Shannon
    """
    
    # collect the output files to merge later
    out_prop_fps = []
    out_prop_JS_fps = []
    # for mulitprocessing
    jobs = []
    # for each indexed datasets files chunk
    for tdx, tables_chunk in enumerate(tables_chunks):


        # tables_chunk = tables_chunk[:3]


        # declare the output file names
        out_prop_fp = '/home/flejzerowicz/netbench/datasets/properties/ck_%s/tab_props.tsv' % tdx
        out_prop_JS_fp = '/home/flejzerowicz/netbench/datasets/properties/ck_%s/JS.tsv' % tdx
        # create folder if need be
        out_prop_dir = os.path.dirname(out_prop_JS_fp)
        if not os.path.isdir(out_prop_dir):
            os.makedirs(out_prop_dir)
        # simply write the names of processed files of the chunk into the chunk folder for tracability
        with open('/home/flejzerowicz/netbench/datasets/properties/ck_%s/files.tsv' % tdx, 'w') as o:
            for table in tables_chunk:
                o.write('%s\n' % table)
                
        # Jensen-Shannon measure (heavy compute)
        #  - hence collect a boolean if some datasets already measured
        addon = 0
        tables_chunk_toJS = tables_chunk
        if os.path.isfile(out_prop_JS_fp):
            # read it and compare the measured datasets to the datasets in the chunk
            out_prop_JS_pd = pd.read_csv(out_prop_JS_fp, header=0, sep='\t')
            # get the files that have not been measured
            tables_chunk_JSed = out_prop_JS_pd['file'].tolist()
            tables_chunk_toJS = [x for x in tables_chunk if x not in tables_chunk_JSed]

        # spawn the process
        p = mp.Process(target=run_prop_catcher, args=(tdx, tables_chunk, tables_chunk_toJS,
                                                      out_prop_fp, out_prop_JS_fp, addon))
        # collect the JS output file
        out_prop_fps.append(out_prop_fp)
        out_prop_JS_fps.append(out_prop_JS_fp)
        jobs.append(p)
        p.start()


        # if tdx == 1:
        #     break


    for job in jobs:
        job.join()

    return out_prop_fps, out_prop_JS_fps

out_prop_fps, out_prop_JS_fps = run(tables_chunks)

tdx ['/home/flejzerowicz/netbench/datasets/weiss_paper/ts_4/txts/table_9.biom.txt', '/home/flejzerowicz/netbench/datasets/weiss_paper/ts_4/txts/table_8.biom.txt', '/home/flejzerowicz/netbench/datasets/weiss_paper/ts_4/txts/table_7.biom.txt']
tdx ['/home/flejzerowicz/netbench/datasets/weiss_paper/ts_4/txts/table_1.biom.txt', '/home/flejzerowicz/netbench/datasets/weiss_paper/ts_3/txts/table_9.biom.txt', '/home/flejzerowicz/netbench/datasets/weiss_paper/ts_3/txts/table_8.biom.txt']
Written: /home/flejzerowicz/netbench/datasets/properties/ck_0/tab_props.tsv
Written: /home/flejzerowicz/netbench/datasets/properties/ck_0/JS.tsv
Written: /home/flejzerowicz/netbench/datasets/properties/ck_1/tab_props.tsv
Written: /home/flejzerowicz/netbench/datasets/properties/ck_1/JS.tsv


In [86]:
def get_JS(out_prop_JS_pd):
    """
    Compute the Jensen-Shannon distances
    Returns them in column-formatted distances, columns:
        - file1
        - file2
        - JS_sym
    """
    # init list to be the matrix
    all_datasets_pairs = []
    # for each pair of datasets
    for it in itertools.combinations_with_replacement(out_prop_JS_pd.index.tolist(),2):
        # sorted B-A to A-B
        i = sorted(it)
        # add a pseudocount to the probabilities
        d1 = out_prop_JS_pd.loc[i[0],].values + 0.00000000001
        d2 = out_prop_JS_pd.loc[i[1],].values + 0.00000000001
        # get the actual distance and collect it
        JS_sym = distance.jensenshannon(d1,d2)
        # JS_sym = ( (scipy.stats.entropy(d1,d2) + scipy.stats.entropy(d2,d1)) / 2 )
        row = i + [JS_sym]
        all_datasets_pairs.append(row)
    # make and write the column-formatted distances
    all_datasets_pairs_pd = pd.DataFrame(all_datasets_pairs, columns = ['file1', 'file2', 'JS_sym'])
    all_datasets_pairs_pd['file1'] = ['_'.join(os.path.splitext(x)[0].split('netbench/datasets/')[-1].split('/')[1:]) for x in all_datasets_pairs_pd['file1'].tolist()]
    all_datasets_pairs_pd['file2'] = ['_'.join(os.path.splitext(x)[0].split('netbench/datasets/')[-1].split('/')[1:]) for x in all_datasets_pairs_pd['file2'].tolist()]
    all_datasets_pairs_pd.to_csv('/home/flejzerowicz/netbench/datasets/properties/JS_table.tsv', index=False, sep='\t')
    return all_datasets_pairs_pd

def write_mains(out_prop_fps, out_prop_JS_fps):
    """
    Concatenate the probability tables and compute the Jensen-Shannon distances between datasets
    """
    # get the JS divergences in long, column format
    main_fp = '/home/flejzerowicz/netbench/datasets/properties/props.tsv'
    out_prop_pd = pd.concat([pd.read_csv(out_prop_fp, header=0, sep='\t') for out_prop_fp in out_prop_fps])
    out_prop_pd.set_index('file', inplace=True)
    out_prop_pd.fillna(0.0).to_csv(main_fp, index=True, sep='\t')
    print('Written:', main_fp)

    # get the JS divergences in long, column format
    main_JS_fp = '/home/flejzerowicz/netbench/datasets/properties/JS.tsv'
    out_prop_JS_pd = pd.concat([pd.read_csv(out_prop_JS_fp, header=0, sep='\t') for out_prop_JS_fp in out_prop_JS_fps])
    out_prop_JS_pd.set_index('file', inplace=True)
    out_prop_JS_pd.fillna(0.0).to_csv(main_JS_fp, index=True, sep='\t')
    print('Written:', main_JS_fp)

    # make symetric matrix
    JS_mat = get_JS(out_prop_JS_pd)
    JS_mat = JS_mat.set_index(['file1', 'file2']).unstack()
    JS_mat.columns = JS_mat.columns.droplevel()
    for i in range(JS_mat.shape[0]):
        for j in range(i, JS_mat.shape[0]):
            JS_mat.iloc[j,i] = JS_mat.iloc[i,j]
    del JS_mat.index.name
    JS_mat_fp = '/home/flejzerowicz/netbench/datasets/properties/JS_mat.tsv'
    print('Written:', JS_mat_fp)
    JS_mat.to_csv(JS_mat_fp, index=True, sep='\t')

write_mains(out_prop_fps, out_prop_JS_fps)

Written: /home/flejzerowicz/netbench/datasets/properties/props.tsv
Written: /home/flejzerowicz/netbench/datasets/properties/JS.tsv
Written: /home/flejzerowicz/netbench/datasets/properties/JS_mat.tsv


# Make Metadata for qiime

In [90]:
datasets_props_fp = '/home/flejzerowicz/netbench/datasets/properties/props.tsv'
datasets_props_pd = pd.read_csv(datasets_props_fp, header=0, sep='\t')

datasets_meta_L = []
for table in tables_list:
    table_path_split = os.path.splitext(table)[0].split('netbench/datasets/')[-1].split('/')
    main_type = table_path_split[0]
    if main_type == 'qiita_papers':
        raw_filt = table_path_split[1]
        if raw_filt == 'filt':
            filt = table_path_split[2]
            dat = table_path_split[3]
        else:
            filt = 'NaN'
            dat = table_path_split[2]
    else:
        raw_filt = 'NaN'
        filt = 'NaN'
        dat = table_path_split[1]
    sam = '_'.join(table_path_split[1:])
    datasets_meta_L.append([sam, main_type, raw_filt, filt, dat, table])
datasets_meta_pd = pd.DataFrame(datasets_meta_L, columns=['sample_name', 'main_type', 'raw_filt', 'filtering', 'dataset', 'file'])
datasets_meta_pd = datasets_meta_pd.merge(datasets_props_pd)
datasets_meta_fp = '/home/flejzerowicz/netbench/datasets/properties/JS_mat_meta.tsv'
datasets_meta_pd.to_csv(datasets_meta_fp, index=False, sep='\t')

In [88]:
!qiime tools import --input-path /home/flejzerowicz/netbench/datasets/properties/JS_mat.tsv --output-path /home/flejzerowicz/netbench/datasets/properties/JS_mat.qza --type DistanceMatrix
!qiime diversity pcoa --i-distance-matrix /home/flejzerowicz/netbench/datasets/properties/JS_mat.qza --o-pcoa /home/flejzerowicz/netbench/datasets/properties/JS_mat_PCoA.qza
!qiime emperor plot --i-pcoa /home/flejzerowicz/netbench/datasets/properties/JS_mat_PCoA.qza \
                    --m-metadata-file /home/flejzerowicz/netbench/datasets/properties/JS_mat_meta.tsv \
                    --o-visualization /home/flejzerowicz/netbench/datasets/properties/JS_mat_PCoA_emp.qzv

Imported /home/flejzerowicz/netbench/datasets/properties/JS_mat.tsv as DistanceMatrixDirectoryFormat to /home/flejzerowicz/netbench/datasets/properties/JS_mat.qza
